In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [4]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install
# pip install catboost

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool

### Область работы 2 (выполнение лучшей модели)

In [ ]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [3]:
# Путь к тренировочному набору
path_train = 'train_diab.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test_diab.csv' # содержит только имя файла, без имен папок

In [ ]:
# Блок обучения модели

In [8]:
df = pd.read_csv('train_diab.csv')

# Определяем категориальные признаки
categorical_features = ['gender', 'ethnicity', 'education_level', 'income_level', 
                        'smoking_status', 'employment_status', 'family_history_diabetes',
                        'hypertension_history', 'cardiovascular_history']

# Кодируем целевую переменную, если она не в числовом формате
if df['diagnosed_diabetes'].dtype == 'object':
    le = LabelEncoder()
    df['diagnosed_diabetes'] = le.fit_transform(df['diagnosed_diabetes'])

# Разделяем на признаки и целевую переменную
X = df.drop('diagnosed_diabetes', axis=1)
y = df['diagnosed_diabetes']

# Удаляем ID из признаков
if 'id' in X.columns:
    X = X.drop('id', axis=1)

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp
)

cat_features_indices = [X.columns.get_loc(col) for col in categorical_features if col in X.columns]

# 7. Создание и обучение CatBoost модели
print("\n6. Обучение CatBoost модели...")

# Определяем параметры модели
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    verbose=100,
    eval_metric='AUC',
    early_stopping_rounds=50,
    cat_features=cat_features_indices,
    class_weights=[1, (y_train.value_counts()[0] / y_train.value_counts()[1])]  # Балансировка классов
)

# Создаем Pool объекты
train_pool = Pool(X_train, y_train, cat_features=cat_features_indices)
val_pool = Pool(X_val, y_val, cat_features=cat_features_indices)

# Обучаем модель
model.fit(
    train_pool,
    eval_set=val_pool,
    plot=True,
    use_best_model=True
)

print("Модель обучена!")


6. Обучение CatBoost модели...


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.6736390	best: 0.6736390 (0)	total: 250ms	remaining: 4m 9s
100:	test: 0.7049565	best: 0.7049565 (100)	total: 24.7s	remaining: 3m 39s
200:	test: 0.7099523	best: 0.7099523 (200)	total: 49.3s	remaining: 3m 15s
300:	test: 0.7149155	best: 0.7149155 (300)	total: 1m 13s	remaining: 2m 50s
400:	test: 0.7185536	best: 0.7185536 (400)	total: 1m 37s	remaining: 2m 25s
500:	test: 0.7204337	best: 0.7204350 (499)	total: 2m 1s	remaining: 2m 1s
600:	test: 0.7216188	best: 0.7216188 (600)	total: 2m 26s	remaining: 1m 37s
700:	test: 0.7225829	best: 0.7225852 (699)	total: 2m 54s	remaining: 1m 14s
800:	test: 0.7233175	best: 0.7233175 (800)	total: 3m 19s	remaining: 49.6s
900:	test: 0.7239729	best: 0.7239729 (900)	total: 3m 44s	remaining: 24.7s
999:	test: 0.7244011	best: 0.7244011 (999)	total: 4m 9s	remaining: 0us

bestTest = 0.7244011473
bestIteration = 999

Модель обучена!


In [ ]:
# Блок предсказания с использованием тестового набора

In [18]:
#test_data = pd.read_csv(path_test)
#data = test_data.copy()
#ids = data['id'].copy()
#data = data.drop('id', axis=1)
test_df = pd.read_csv(path_test)

def preprocess_test_data(test_df):

    # Сохраняем ID отдельно
    ids = test_df['id'].copy()
    
    # Копируем данные и удаляем id
    processed_data = test_df.drop('id', axis=1).copy()
    
    return processed_data, ids


def predict_and_save_results(test_df, model, output_path='submission.csv'):
 
    print(f"🚀 Начало предсказания для {len(test_df)} записей...")
    
    # Предобработка данных
    processed_data, ids = preprocess_test_data(test_df)
    print(f"✓ Данные подготовлены: {processed_data.shape[0]} записей, {processed_data.shape[1]} признаков")
    
    # Определяем категориальные признаки
    cat_features = ['gender', 'ethnicity', 'education_level', 'income_level', 
                   'smoking_status', 'employment_status', 'family_history_diabetes',
                   'hypertension_history', 'cardiovascular_history']
    
    # Создаем Pool для CatBoost
    cat_indices = [processed_data.columns.get_loc(col) for col in cat_features 
                   if col in processed_data.columns]
    
    test_pool = Pool(processed_data, cat_features=cat_indices)
    
    # Выполняем предсказание
    print("🔮 Выполнение предсказаний...")
    predictions = model.predict(test_pool)
    
    # Создаем DataFrame с результатами
    results = pd.DataFrame({
        'id': ids.values,
        'diagnosed_diabetes': predictions.astype(int)
    })
    
    # Сохраняем в CSV файл
    results.to_csv(output_path, index=False)
    print(f"💾 Результаты сохранены в файл: {output_path}")
    print(f"   Всего записей: {len(results)}")
    print(f"   Диабет предсказан для {results['diagnosed_diabetes'].sum()} записей")
    print(f"   Процент предсказанного диабета: {results['diagnosed_diabetes'].mean()*100:.1f}%")
    
    return results

preprocess_test_data(test_df)
predict_and_save_results(test_df, model, output_path='submission.csv')

🚀 Начало предсказания для 300000 записей...
✓ Данные подготовлены: 300000 записей, 24 признаков
🔮 Выполнение предсказаний...
💾 Результаты сохранены в файл: submission.csv
   Всего записей: 300000
   Диабет предсказан для 140195 записей
   Процент предсказанного диабета: 46.7%


,id,diagnosed_diabetes
0,700000,0
1,700001,1
2,700002,1
3,700003,0
4,700004,1
...,...,...
299995,999995,1
299996,999996,1
299997,999997,0
299998,999998,1


In [14]:
# Вектор предсказанных значений  y_predict полученый на основане тестового набора
#y_predict = model.predict(test_data)
#y_predict